# Lec 10.2 plotting shapefiles

## Shape File Demo

---

Creator: Riley Brady
Contact: riley.brady@colorado.edu

---

Download LME shapefiles here: http://lme.edc.uri.edu/index.php/digital-data/113-lme-polygon-boundaries

---

Package requirements:
* numpy
* geopandas
* matplotlib
* cartopy
* descartes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import geopandas as gpd

### E1. Make sure the above packages all load in your environment

all good? continue...

## 1. Loading shapefiles in geopandas

First we will load in a shapefile and look at the information in it

In [ ]:
# function to load shapfiles in as a geopandas object
def load_shape_file(filepath):
    """Loads the shape file desired to mask a grid.
    Args:
        filepath: Path to *.shp file
    """
    shpfile = gpd.read_file(filepath)
    return shpfile

In [ ]:
shp = load_shape_file('/Users/chha5666/Documents/Clim_var/Fisheries_metric/Data/LME66/LMEs66.shp')
shp.head() # .head gives you the first few lines, aka shapes

In [ ]:
shp # this gives you all the shapes

# watch out, #4 is a multi-polygon (a few separte polygons making up one shape)

In [ ]:
# another way to view the same data (not as pretty)
print(shp)

### E2. What type of object is shp?

## 2. Plotting shapefiles using basemap

Basemap is a mapping package in python that has been around for a while. It is being replaced by Cartopy, but the latter does not have all of the functionality yet. Your instructor uses it often to plot model data

https://matplotlib.org/basemap/

https://basemaptutorial.readthedocs.io/en/latest/



In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
# Basic plot in basemap

ax = plt.figure(figsize=(16,20), facecolor = 'w')

# limits of plot
limN, limS, limE, limW = 84.,-80.,180,-180

#m = Basemap(projection='hammer',lon_0=0)
m = Basemap(projection='cyl', llcrnrlon=limW, \
      urcrnrlon=limE, llcrnrlat=limS, urcrnrlat=limN, resolution='c')
m.drawcoastlines()
m.drawmapboundary()
m.fillcontinents(color='#BDA973', lake_color='#BDA973');

In [ ]:
# add in some lat lon lines

ax = plt.figure(figsize=(16,20), facecolor = 'w')

# limits of plot
limN, limS, limE, limW = 84.,-80.,180,-180

#m = Basemap(projection='hammer',lon_0=0)
m = Basemap(projection='cyl', llcrnrlon=limW, \
      urcrnrlon=limE, llcrnrlat=limS, urcrnrlat=limN, resolution='c')
m.drawcoastlines()
m.drawmapboundary()
m.fillcontinents(color='#BDA973', lake_color='#BDA973');

parallels = np.arange(-90.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(-180.,180.,20.)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10);

### E3. One at a time: a) Change the color of the land. b) Make the lakes blue. c) Change the resolution of the map. d) Zoom into Gulf of Mexico and make the lat lon grid better resolution for this region.


see https://matplotlib.org/examples/color/named_colors.html for all the color names


alternately you can use hex colors (which is what is used above): 

https://www.color-hex.com/color-palettes/

http://colorbrewer2.org/#type=sequential&scheme=YlGnBu&n=6


### Plotting the shapefile

In [ ]:
# for plotting the entire shapefile, we want the file name without the .shp extension
sppath= '/Users/chha5666/Documents/Clim_var/Fisheries_metric/Data/LME66/LMEs66'

In [ ]:
# plot in basemap with LME boundaries

ax = plt.figure(figsize=(16,20), facecolor = 'w')

# limits of plot
limN, limS, limE, limW = 84.,-80.,180,-180


#m = Basemap(projection='hammer',lon_0=0)
m = Basemap(projection='cyl', llcrnrlon=limW, \
      urcrnrlon=limE, llcrnrlat=limS, urcrnrlat=limN, resolution='c')
m.drawcoastlines()
m.drawmapboundary()
m.fillcontinents(color='#d8b365', lake_color='w')

m.readshapefile(sppath, 'LME') # the second argument is a name for the shapefile data inside the shapefile 

# plot all the shapes in the shapefile (magic black box code off of stack exchange):
for info, shape in zip(m.LME_info, m.LME):
        x, y = zip(*shape) 
        m.plot(x, y, marker=None,color='k', linewidth = '2')       

### E4. Change the color and the linewidth of the shapefile lines. 

## 3. Plotting shapfile polygons using cartopy and descartes

Cartopy is replacing Basemap as the map plotting package for python. It doesn't have all the functionality yet, but it is what the community is moving to so it's worth putting the time in to learn

https://scitools.org.uk/cartopy/docs/latest/index.html

https://scitools.org.uk/cartopy/docs/latest/gallery/index.html



In [ ]:
# function to pick out one shape from the shapefile
def select_shape(shpfile, category, name):
    """Select the submask of interest from the shapefile.
    Args:
        geopandas shpfile object: (from *.shp) loaded through `load_shape_file`
        category: (str) header of shape file from which to filter shape.
            (Run print(shpfile) to see options)
        name: (str) name of shape relative to category.
        plot: (optional bool) if True, plot the polygon that will be masking.
    Returns:
        shapely polygon
    Example:
        from esmask.mask import load_shape_file, select_shape
        LME = load_shape_file('LMEs.shp')
        CalCS = select_shape(LME, 'LME_NAME', 'California Current')
    """
    s = shpfile
    polygon = s[s[category] == name]
    polygon = polygon.geometry[:].unary_union #magic black box off of stack exchange (should paste link), concatinating polygons
    return polygon

In [ ]:
# note the is using the "LME_NAME" category from the geopandas shapefile object
CalCS_shp = select_shape(shp, 'LME_NAME', 'California Current')
CalCS_shp

### E5. What kind of object is CalCS_shp?

### E6. Make the same plot as above for a) the Gulf of Alaska and b) Labrador - Newfoundland LMEs

### Plot a single polygon in the shapefile as a patch on a map

In [ ]:
# Riley's functions for making pretty map plots


def lat_lon_formatter(ax):
    """
    Creates nice latitude/longitude labels
    for maps
    """
    lon_formatter = cticker.LongitudeFormatter()
    lat_formatter = cticker.LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.tick_params(labelsize=16)
    
# the below function uses lat_lon_formatter above
def set_up_map(ax, x0, x1, y0, y1):
    """
    Adds coastline, etc.
    
    x0, x1: longitude bounds to zoom into
    y0, y1: latitude bounds to zoom into
    """
    # set up land overlay
    ax.add_feature(cfeature.LAND, facecolor='k')
    
    # zoom in on region of interest
    ax.set_extent([x0, x1, y0, y1])
    
    # set nicer looking ticks
    ax.set_xticks(np.arange(x0, x1, 10), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(y0, y1, 10), crs=ccrs.PlateCarree())
    lat_lon_formatter(ax)

## California Current Demo

First plot a basic map of the US west coast using cartopy

In [ ]:
f, ax = plt.subplots(ncols=2, figsize=(10,5),
                     subplot_kw=dict(projection=ccrs.PlateCarree())) # this last bit is the map projection from Cartopy
set_up_map(ax[0], -140, -107, 20, 50)
set_up_map(ax[1], -140, -107, 20, 50)

### E7. How would you change the color of the land in the above? Try it

### Plot the California Current Large Marine Ecosystem polygon two ways

In [ ]:
from descartes import PolygonPatch

Descartes is an extension of shapely that allows one to plot shapefile objects as polygon patches in matplotlib plots

https://pypi.org/project/descartes/

In [ ]:
# look at the help files for PolygonPatch
help(PolygonPatch)

# here alpha is the transparency

In [ ]:
f, ax = plt.subplots(ncols=2, figsize=(10,5),
                     subplot_kw=dict(projection=ccrs.PlateCarree()))
set_up_map(ax[0], -140, -107, 20, 50)
set_up_map(ax[1], -140, -107, 20, 50)

# add shapefile to map
ax[0].add_patch(PolygonPatch(CalCS_shp, fc='#add8e6')) # Note PolygonPatch takes all the matplotlib commands
# some other attributes to play around with
ax[1].add_patch(PolygonPatch(CalCS_shp, fc='None', ec='r', linewidth=2,
                             linestyle=':')) # fc= "None is needed to make the inside of the shape transparent"

In [ ]:
# let's plot them on top of each other

fig= plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
set_up_map(ax, -140, -107, 20, 50)

# plot the shapefile in blue with a red dotted boundary
ax.add_patch(PolygonPatch(CalCS_shp, fc='None', ec='r', linewidth=2,
                             linestyle=':'))
ax.add_patch(PolygonPatch(CalCS_shp, fc='#add8e6', ec = 'None', alpha = 1, zorder = 0))

### E8. Change zorder to 10 in the last line above, and omit the ec statement. What changes? What happened?

### E9. Find the four East Pacific LMES and plot them in different colors on the same map